In [ ]:
import pandas as pd
import numpy as np
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from autogluon.common import space

# auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()

In [ ]:
print("loading data")
# Prepare data
train = TabularDataset("../data/mushrooms/train.csv")
test = TabularDataset("../data/mushrooms/test.csv")

XGB = {
    "eta": space.Real(0.01, 0.2, default=0.1),
    "max_depth": space.Int(
        1, 3, default=2
    ),  # Adjusted range to be realistic for max_depth
    "min_child_weight": space.Int(
        1, 30, default=1
    ),  # Adjusted range to be realistic for min_child_weight
    "gamma": space.Real(0.01, 0.2, default=0.1),
    "subsample": space.Real(0.01, 0.2, default=0.1),
    "colsample_bytree": space.Real(0, 1, default=0.5),
    "lambda": space.Real(0.01, 0.2, default=0.1),
    "alpha": space.Real(0.01, 0.2, default=0.1),
}
hyperparameters = {  # hyperparameters of each model type
    "XGB": XGB,
}
time_limit = 2 * 60  # train various models for ~2 min
num_trials = (
    5  # try at most 5 different hyperparameter configurations for each type of model
)
search_strategy = (
    "auto"  # to tune hyperparameters using random search routine with a local scheduler
)

hyperparameter_tune_kwargs = (
    {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
        "num_trials": num_trials,
        "scheduler": "local",
        "searcher": search_strategy,
    }
)  # Refer to TabularPredictor.fit docstring for all valid values
# Training
predictor = TabularPredictor(
    label="class",
    problem_type="binary",
    eval_metric="log_loss",
).fit(
    train_data=train,
    time_limit=time_limit,
    presets=["best_quality"],
    num_gpus=1,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)